In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

all_imgs = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms=transforms.Compose([
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizentalFlip(),
    transforms.ToTensor(),
]))

loader = torch.utils.data.DataLoader(all_imgs, batch_size=batch_size, shuffle=True)


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizentalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                   batch_size=batch_size, shuffle=True,
                                                   num_workers=4) for x in ['train', 'val']}
device = torch.device('cuda' if torch.cuda.is_avaliable() else 'cpu')

def set_parameter_requires_grad(model, feature_extract = True):
    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False
def initialize_mode(model_name, num_classes, feature_extract, use_pretrained = True):
    if model_name == 'resnet':
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    else:
        print('Model not implemented')
        return None, None
    return model_ft, input_size

model_ft, input_size = initialize_mode('resnet', 2, True, True)
print(model_ft)

SyntaxError: invalid syntax (<ipython-input-1-08624f484b32>, line 9)

In [2]:
def train_model(model, dataloaders, loss_fn, optimizer, num_epochs = 5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.
    val_acc_history = []
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            running_loss = 0.
            running_corrects = 0.
            running_c
            if phase == 'train':
                model.train()
            else:
                model.eval()
            for inputs, labels in dataloaders(phase):
                inputs, labels = inputs.to(device), labels.to(device)
                with torch.autograd.set_grad_enabled(phase='train'):
                    outputs = model(inputs) # bsize * 2
                    loss = loss_fn(outputs, labels)
                preds = outputs.argmax(dim=1)
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds.view(-1) == labels.view(-1)).item()
            epoch_loss = running_loss / len(dataloaders(phase).dataset)
            epoch_acc = running_corrects / len(dataloaders(phase).dataset)
            
            print('Phse {} loss: {}, acc: {}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'val' and epoch_acc > best_acc:
                best_model_wts = copy.deepcopy(model.state_dict())
                best_acc = epoch_acc
            if phase == 'val':
                val_acc_history.append(epoch_acc)
    model.load_state_dict(best_model_wts)
    return model, val_loss_history

In [6]:
print('模型训练')

模型训练


In [7]:
model_ft = model_ft.to(device)
# optimizer = torch.optim.SGC(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=0.001, momentum=0.9)
optimizer = torch.optim.SGC(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

_, ohist = train_model(model_ft, dataloaders_dict, loss_fn, optimizer)

NameError: name 'model_ft' is not defined

In [ ]:
model_scratch, _ = initialize_model(model_name, num_classes, feature_extract = False, use_pretrained = False)

optimizer = torch.optim.SGC(filter(lambda p: p.requires_grad, model_scratch.parameters()), lr=0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

_, scratch_hist = train_model(model_scratch, dataloaders_dict, loss_fn, optimizer)